# Main File :

# Importing Packages and Modules 

In [1]:
# my model is stored in the form of json file so to get the model from json file i am using the module model_from_json

from tensorflow.keras.models import model_from_json
import numpy as np
import tensorflow as tf
import cv2

# enabling tensor flow 1 

config=tf.compat.v1.ConfigProto()
session=tf.compat.v1.Session(config=config)


# FacialExpressionModel Class

In [2]:
# this class is used to predict output(facial expression ) corrosponds to the face image
# i will pass  two parametere to this class 
# first is a json file which contain my model archietecture 
# second is model_weight_file which contain the weights which i used to train my model

class FacialExpressionModel(object):
    # Emotion list 
    EMOTION_LIST=["Angry","Disgust","Fear","Happy","Neutral","Sad","Surprise"]
    
    # this will take two parameters as input ,one is a json_file(model archietecture) and other one is weights
    
    def __init__ (self , model_json_file , model_weights_file):
        
        with open(model_json_file, "r") as json_file:
             loaded_model_json=json_file.read()
        
        # making the model from json_file
        self.loaded_model=model_from_json(loaded_model_json)
        
        # loading weights into the model
        self.loaded_model.load_weights(model_weights_file)
        
        # making predict function
        self.loaded_model._make_predict_function()
    
    # function used to predict emotion
    def predict_emotion(self, img ):
        
        # self.preds is a list where each value of list indicate chance of that particular index class
        self.preds=self.loaded_model.predict(img)

        # using np.argmax to find the index of maximum value and then returning the emotion corrosponds to that emotion
        return FacialExpressionModel.EMOTION_LIST[np.argmax(self.preds)] 


# Capturing images and predicting output

In [5]:
#using harcascade classifier to detect face in frame

# creating object of CascadeClassifier
face_classifier = cv2.CascadeClassifier("C:/Users/shivamGarg/AppData/Local/Programs/Python/Python37/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml")

# creating object of FacialExpressionModel , whose predict_emotion function is used to get the expression .
# passing model_json (json file which contain model archietecture) and facial_weight_model ( weight of model )

my_model = FacialExpressionModel("model_json","model_weights.h5")

# using default webCam connected to pc
video = cv2.VideoCapture(0) 

while True:
    
    # reading each frame one by one
    _, img= video.read()
    
    # converting the image to grey scale 
    grey_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # detecting all faces in image
    faces=face_classifier.detectMultiScale(grey_img,1.2,5)
    
    # if we found a face in image
    if faces is not ():
        
        # Draw rectangles around each face and predicting output
        for(x,y,w,h) in faces:
            
            # return a list of pixel value for face 
            fc = grey_img[y:y+h,x:x+w]
            
            # resizing the list into required image shape
            face=cv2.resize(fc,(48,48))
            
            # use my_model to predict expression
            pred = my_model.predict_emotion(face[np.newaxis,:,:,np.newaxis])
            
            # putting the predicted value text on image
            cv2.putText(img,pred,(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,0),1)
            
            # drawing rectangle around the face in image
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        
        # showing the image 
        cv2.imshow("img",img)
        
    else :
        print("Face not found")
        cv2.imshow("img",img)
   
    # waitkey is used to stop the model
    k=cv2.waitKey(1)
    # 27 corrosponds to Esc 
    if k==27:
        break

# destroying all windows 
cv2.destroyAllWindows()

# releasing the camera
video.release()



Face not found
Face not found
Face not found
Face not found
